# 03. Tokenizing

## Setting

In [1]:
import pandas as pd 
import numpy as np
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# rc('font', family='AppleGothic') # Mac 
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
data_path = "data/naver_review_preprocessing.csv"
data = pd.read_csv(data_path, parse_dates=["date"], encoding="CP949")
data.head(5)

,reviewer,review,date,weekday
0,shooooooo,종류도 알차게 많고 다 맛있어요,2024-06-07,금요일
1,쥴리08,커피랑 브런치크림파스타 먹었어요 오늘따라 파스타가 불어서나왔더라구요 역시 신라 커피...,2024-05-23,목요일
2,illiiilillil,직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금...,2024-05-19,일요일
3,DEAN KIL,비싸지만 좋은곳,2024-05-04,토요일
4,이진욱쨩,항상 친절하시고 음식도맛있네요,2024-04-21,일요일


In [3]:
data.shape

(317, 4)

## 1. 형태소 분석

* 불용어, 신조어, 가중치 등 여러 기능들이 포함되어 있는 Kiwi 형태소 분석기를 선택하여 분석

In [4]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
kiwi = Kiwi()
stopwords = Stopwords()

In [5]:
review_data = data.copy()
review_data.head(5)

,reviewer,review,date,weekday
0,shooooooo,종류도 알차게 많고 다 맛있어요,2024-06-07,금요일
1,쥴리08,커피랑 브런치크림파스타 먹었어요 오늘따라 파스타가 불어서나왔더라구요 역시 신라 커피...,2024-05-23,목요일
2,illiiilillil,직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금...,2024-05-19,일요일
3,DEAN KIL,비싸지만 좋은곳,2024-05-04,토요일
4,이진욱쨩,항상 친절하시고 음식도맛있네요,2024-04-21,일요일


* 의미가 있다고 생각하는 체언(N), 용언(V)만 사용할 예정

In [14]:
review_json = []

allow_tag = ["N", "V"]
for review in review_data["review"]:
    review_dict = {}

    tokens = kiwi.tokenize(review, stopwords=stopwords)
    tag_dict = {"N": [], "V": []}
    for token in tokens:
        if token.tag[0] in allow_tag:
            tag_dict[token.tag[0]].append(token.form)
    
    review_dict["review"] = review
    review_dict["tokens"] = [str(token) for token in tokens]
    review_dict["N"] = " ".join(tag_dict["N"])
    review_dict["V"] = " ".join([x+"다" for x in tag_dict["V"]])

    review_json.append(review_dict)

In [8]:
analyze_data = pd.json_normalize(review_json).drop(columns="tokens")
analyze_data.head(5)

,review,N,V
0,종류도 알차게 많고 다 맛있어요,종류,알차다 많다 맛있다
1,커피랑 브런치크림파스타 먹었어요 오늘따라 파스타가 불어서나왔더라구요 역시 신라 커피...,커피 브런치 크림 파스타 파스타 신라 커피,먹다 불다 나오다 맛있다
2,직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금...,직원 친절 음식 맛 대한민국 호텔 부페 넘버원 뭐 조금 입장 융통 거 일류 구색 거,이다 좋다 아쉽다 약하다
3,비싸지만 좋은곳,곳,비싸다 좋다
4,항상 친절하시고 음식도맛있네요,친절 음식,맛있다


In [90]:
total_data = pd.merge(review_data, analyze_data, how="inner", on="review")
total_data.head(5)

,reviewer,review,date,weekday,N,V
0,shooooooo,종류도 알차게 많고 다 맛있어요,2024-06-07,금요일,종류,알차다 많다 맛있다
1,쥴리08,커피랑 브런치크림파스타 먹었어요 오늘따라 파스타가 불어서나왔더라구요 역시 신라 커피...,2024-05-23,목요일,커피 브런치 크림 파스타 파스타 신라 커피,먹다 불다 나오다 맛있다
2,illiiilillil,직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금...,2024-05-19,일요일,직원 친절 음식 맛 대한민국 호텔 부페 넘버원 뭐 조금 입장 융통 거 일류 구색 거,이다 좋다 아쉽다 약하다
3,DEAN KIL,비싸지만 좋은곳,2024-05-04,토요일,곳,비싸다 좋다
4,이진욱쨩,항상 친절하시고 음식도맛있네요,2024-04-21,일요일,친절 음식,맛있다


## ⭐️ 전처리 데이터 저장

In [15]:
import json 

with open("./data/naver_review_tokenizing.json", "w", encoding="utf-8") as json_file:
    json.dump(review_json, json_file, indent=4, ensure_ascii=False)

In [126]:
total_data = total_data[["reviewer","date","weekday","review","N","V"]]
total_data.to_csv("./data/naver_review_tokenizing.csv", index=False, encoding="CP949")
total_data.head(5)

,reviewer,date,weekday,review,N,V
0,shooooooo,2024-06-07,금요일,종류도 알차게 많고 다 맛있어요,종류,알차다 많다 맛있다
1,쥴리08,2024-05-23,목요일,커피랑 브런치크림파스타 먹었어요 오늘따라 파스타가 불어서나왔더라구요 역시 신라 커피...,커피 브런치 크림 파스타 파스타 신라 커피,먹다 불다 나오다 맛있다
2,illiiilillil,2024-05-19,일요일,직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금...,직원 친절 음식 맛 대한민국 호텔 부페 넘버원 뭐 조금 입장 융통 거 일류 구색 거,이다 좋다 아쉽다 약하다
3,DEAN KIL,2024-05-04,토요일,비싸지만 좋은곳,곳,비싸다 좋다
4,이진욱쨩,2024-04-21,일요일,항상 친절하시고 음식도맛있네요,친절 음식,맛있다
